# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [1]:
# Import panda
import os, sys
import pandas as pd
import numpy as np
import csv
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.
DATA_DIRECTORY = './Data/ebola/'

In [84]:
def get_csv_in_chrono_order(folder_path):
    filenames = pd.Series(os.listdir(folder_path)).sort_values()
    filenames.index = range(filenames.shape[0])
    return filenames

def get_interesting_values(folder_path,filenames,interesting_fields,interesting_columns,field_column_name):
    dfs = []
    for fname in filenames:
        new_df = pd.read_csv(folder_path + fname,usecols=interesting_columns)
        for field in interesting_fields:
            dfs.append( new_df.loc[new_df[field_column_name] == field])
    return pd.concat(dfs)
        

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [85]:
data_folder = DATA_DIRECTORY + 'guinea_data/'
filenames_guinea = get_csv_in_chrono_order(data_folder)

print((pd.read_csv(data_folder + filenames_guinea[1],usecols=['Date','Description','Totals'])))
filenames_guinea.count






          Date                                      Description Totals
0   2014-08-26                            New cases of suspects     18
1   2014-08-26                           New cases of probables    NaN
2   2014-08-26                           New cases of confirmed     10
3   2014-08-26                Total new cases registered so far     28
4   2014-08-26                          Total cases of suspects     30
5   2014-08-26                         Total cases of probables    141
6   2014-08-26                         Total cases of confirmed    490
7   2014-08-26     Cumulative (confirmed + probable + suspects)    661
8   2014-08-26                  Total suspected non-class cases    518
9   2014-08-26                            New deaths registered      5
10  2014-08-26                         Total deaths of suspects      2
11  2014-08-26                        Total deaths of probables    141
12  2014-08-26                        Total deaths of confirmed    292
13  20

<bound method Series.count of 0     2014-08-04.csv
1     2014-08-26.csv
2     2014-08-27.csv
3     2014-08-30.csv
4     2014-08-31.csv
5     2014-09-02.csv
6     2014-09-04.csv
7     2014-09-07.csv
8     2014-09-08.csv
9     2014-09-09.csv
10    2014-09-11.csv
11    2014-09-14.csv
12    2014-09-16.csv
13    2014-09-17.csv
14    2014-09-19.csv
15    2014-09-21.csv
16    2014-09-22.csv
17    2014-09-23.csv
18    2014-09-24.csv
19    2014-09-26.csv
20    2014-09-30.csv
21    2014-10-01.csv
dtype: object>

In [86]:
fields = ['New cases of suspects','New cases of probables','New cases of confirmed',
          'New deaths registered','Total deaths of suspects',
         'Total deaths of probables','Total deaths of confirmed']
new_cases = get_interesting_values(data_folder,filenames_guinea,fields,['Date','Description','Totals'],'Description')
new_cases = new_cases.set_index('Date')
colsname = new_cases['Description'].unique()

series = []
for elem in colsname:
    series.append(new_cases.loc[(new_cases['Description'] == elem)])
    series[len(series)-1] = series[len(series)-1].drop(['Description'],axis=1)
    series[len(series)-1] = series[len(series)-1].rename(columns={'Totals':elem})
    
    
pd.concat(series,axis=1,join='outer')

,New cases of suspects,New cases of probables,New cases of confirmed,Total deaths of suspects,Total deaths of probables,Total deaths of confirmed,New deaths registered
2014-08-04,5,0,4,2,133,228,NaN
2014-08-26,18,NaN,10,2,141,292,5
2014-08-27,12,0,10,2,142,294,2
2014-08-30,15,0,9,2,142,324,5
2014-08-31,9,8,29,2,150,337,3
2014-09-02,11,0,14,2,150,349,5
2014-09-04,13,0,17,3,151,368,5
2014-09-07,5,0,11,4,151,402,4
2014-09-08,5,0,11,4,151,402,4
2014-09-09,9,0,7,4,151,410,7


In [87]:
ordered_dates = [x[0:10] for x in filenames_guinea]
ordered_dates = pd.to_datetime(ordered_dates,format='%Y-%m-%d')
time_span = ordered_dates[ordered_dates.shape[0]-1] - ordered_dates[0]

# Liberia

In [88]:
data_folder = DATA_DIRECTORY + 'liberia_data/'
filenames_liberia = get_csv_in_chrono_order(data_folder)

print(pd.read_csv(data_folder + filenames_liberia[21],usecols=['Date','Variable','National']))
filenames_liberia.count

         Date                                           Variable  National
0   8/15/2014                                Specimens collected      70.0
1   8/15/2014                      Specimens pending for testing      82.0
2   8/15/2014                             Total specimens tested      31.0
3   8/15/2014                              Newly reported deaths      17.0
4   8/15/2014                   Total death/s in confirmed cases     172.0
5   8/15/2014                    Total death/s in probable cases     200.0
6   8/15/2014                   Total death/s in suspected cases      83.0
7   8/15/2014  Total death/s in confirmed, probable, suspecte...     455.0
8   8/15/2014  Case Fatality Rate (CFR) - Confirmed & Probabl...       NaN
9   8/15/2014                            Newly reported contacts      59.0
10  8/15/2014                              Total contacts listed    1777.0
11  8/15/2014                          Currently under follow-up    1265.0
12  8/15/2014            

<bound method Series.count of 0          2014-06-16.csv
1          2014-06-17.csv
2          2014-06-22.csv
3          2014-06-24.csv
4          2014-06-25.csv
5          2014-06-28.csv
6          2014-06-29.csv
7          2014-07-01.csv
8          2014-07-02.csv
9          2014-07-03.csv
10         2014-07-07.csv
11         2014-07-08.csv
12         2014-07-10.csv
13         2014-07-13.csv
14         2014-07-17.csv
15         2014-07-20.csv
16         2014-07-24.csv
17         2014-07-26.csv
18         2014-08-02.csv
19         2014-08-04.csv
20         2014-08-12.csv
21         2014-08-15.csv
22         2014-08-17.csv
23         2014-08-18.csv
24         2014-08-20.csv
25         2014-08-25.csv
26         2014-08-28.csv
27         2014-09-01.csv
28         2014-09-02.csv
29         2014-09-03.csv
             ...         
70    2014-10-24-v162.csv
71    2014-10-25-v163.csv
72    2014-10-28-v166.csv
73    2014-10-29-v167.csv
74    2014-10-30-v168.csv
75    2014-10-31-v169.csv
76    20

In [91]:
fields = ['New Case/s (Suspected)','New Case/s (Probable)','New case/s (confirmed)',
         'Total death/s in confirmed cases', 'Total death/s in probable cases',
          'Total death/s in suspected cases', 'Newly reported deaths']
new_cases = get_interesting_values(data_folder,filenames_liberia,fields,['Date','Variable','National'],'Variable')
new_cases = new_cases.set_index('Date')
colsname = new_cases['Variable'].unique()

series = []
for elem in colsname:
    series.append(new_cases.loc[(new_cases['Variable'] == elem)])
    series[len(series)-1] = series[len(series)-1].drop(['Variable'],axis=1)
    series[len(series)-1] = series[len(series)-1].rename(columns={'National':elem})
    
series   
pd.concat(series,axis=1,join='inner')

,New Case/s (Suspected),New Case/s (Probable),New case/s (confirmed),Total death/s in confirmed cases,Total death/s in probable cases,Total death/s in suspected cases,Newly reported deaths
Date,,,,,,,
6/16/2014,2.0,1.0,1.0,8.0,6.0,2.0,2.0
6/17/2014,2.0,0.0,0.0,8.0,6.0,2.0,0.0
6/22/2014,5.0,0.0,5.0,16.0,8.0,1.0,4.0
6/24/2014,2.0,0.0,4.0,18.0,8.0,6.0,4.0
6/25/14,4.0,1.0,2.0,20.0,8.0,9.0,3.0
6/28/2014,2.0,6.0,1.0,26.0,10.0,13.0,1.0
6/29/2014,0.0,0.0,2.0,26.0,10.0,13.0,0.0
7/1/2014,2.0,0.0,2.0,30.0,17.0,14.0,5.0
7/2/2014,1.0,3.0,0.0,32.0,20.0,14.0,5.0


# Sierra Léon

In [95]:
data_folder = DATA_DIRECTORY + 'sl_data/'
filenames_sl = get_csv_in_chrono_order(data_folder)

print(pd.read_csv(data_folder + filenames_sl[21],usecols=['date','variable','National']))
filenames_sl.count




          date                variable National
0   2014-09-02              population  6348350
1   2014-09-02             new_noncase       21
2   2014-09-02           new_suspected        4
3   2014-09-02            new_probable        0
4   2014-09-02           new_confirmed       31
5   2014-09-02             cum_noncase      981
6   2014-09-02           cum_suspected       78
7   2014-09-02            cum_probable       37
8   2014-09-02           cum_confirmed     1146
9   2014-09-02         death_suspected       11
10  2014-09-02          death_probable       37
11  2014-09-02         death_confirmed      399
12  2014-09-02                     cfr     34.8
13  2014-09-02       etc_new_admission      NaN
14  2014-09-02  etc_currently_admitted      NaN
15  2014-09-02       etc_cum_admission      NaN
16  2014-09-02          etc_new_deaths      NaN
17  2014-09-02          etc_cum_deaths      NaN
18  2014-09-02      etc_new_discharges      NaN
19  2014-09-02      etc_cum_discharges  

<bound method Series.count of 0       2014-08-12-v77.csv
1       2014-08-13-v78.csv
2       2014-08-14-v79.csv
3       2014-08-15-v80.csv
4       2014-08-16-v81.csv
5       2014-08-17-v82.csv
6       2014-08-18-v83.csv
7       2014-08-19-v84.csv
8       2014-08-20-v85.csv
9       2014-08-21-v86.csv
10      2014-08-22-v87.csv
11      2014-08-23-v88.csv
12      2014-08-24-v89.csv
13      2014-08-25-v90.csv
14      2014-08-26-v91.csv
15      2014-08-27-v92.csv
16      2014-08-28-v93.csv
17      2014-08-29-v94.csv
18      2014-08-30-v95.csv
19      2014-08-31-v96.csv
20      2014-09-01-v97.csv
21      2014-09-02-v98.csv
22      2014-09-03-v99.csv
23     2014-09-05-v101.csv
24     2014-09-06-v102.csv
25     2014-09-07-v103.csv
26     2014-09-08-v104.csv
27     2014-09-09-v105.csv
28     2014-09-10-v106.csv
29     2014-09-11-v107.csv
              ...         
73     2014-10-27-v153.csv
74     2014-10-28-v154.csv
75     2014-10-30-v156.csv
76     2014-10-31-v157.csv
77     2014-11-01-v158.cs

In [97]:
fields = ['new_suspected','new_probable','new_confirmed',
         'death_suspected', 'death_probable',
          'death_confirmed', 'etc_new_deaths']
new_cases = get_interesting_values(data_folder,filenames_sl,fields,['date','variable','National'],'variable')
new_cases = new_cases.set_index('date')
colsname = new_cases['variable'].unique()

series = []
for elem in colsname:
    series.append(new_cases.loc[(new_cases['variable'] == elem)])
    series[len(series)-1] = series[len(series)-1].drop(['variable'],axis=1)
    series[len(series)-1] = series[len(series)-1].rename(columns={'National':elem})
    
series   
pd.concat(series,axis=1,join='inner')

,new_suspected,new_probable,new_confirmed,death_suspected,death_probable,death_confirmed,etc_new_deaths
date,,,,,,,
2014-08-12,10,1,11,5,34,264,NaN
2014-08-13,3,1,15,5,34,273,NaN
2014-08-14,0,2,13,5,34,280,NaN
2014-08-15,6,1,10,5,34,287,NaN
2014-08-16,3,0,18,5,34,297,NaN
2014-08-17,1,0,2,5,34,305,NaN
2014-08-18,40,15,5,5,34,312,NaN
2014-08-19,16,3,9,5,34,320,NaN
2014-08-20,1,0,4,5,34,322,NaN


## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here